**Goals**

Crowdsourcing annotations has become a fundamental aspect of NLP research. The goal of this hands-on exercise is to explore the ethical implications of soliciting crowdsourced data, specifically social biases that may emerge when asking for generated sentences.

**Overview**

In this exercise, you will perform a “bias audit” of an NLP dataset produced by crowdsourcing. You will attempt to measure the presence of social stereotypes in this dataset that may have harmful effects if used to train classifiers in downstream tasks.

We will use pointwise mutual information (PMI) to find which associations are being made with identity labels. PMI can be used as a measure of word association in a corpus, i.e. how frequently two words co-occur above what might just be expected based on their frequencies.

See the [PMI Wikipedia page](https://en.wikipedia.org/wiki/Pointwise_mutual_information) for more details. Here we use PMI to measure which words co-occur with labels for identities. This allows us to see associations that may perpetuate stereotypes.

After this analysis, you will present specific examples from the data that you speculate could be particularly biased and problematic.

Assignment design credits: [11-830 Computational Ethics in NLP Course](https://maartensap.com/11830/Spring2024/hw1.html)

**Load and prepare dataset**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/MyDrive/Colab Notebooks/ss-data"

Mounted at /content/drive
identity_labels.txt  snli_1.0_train.jsonl


In [2]:
#List of identity labels (based on Rudinger et al. 2017)
#More comprehensive list can be obtained from Jain et al. 2024
with open("/content/drive/MyDrive/Colab Notebooks/ss-data/identity_labels.txt", 'r') as f:
    identity_labels = f.read().split("\n")
print(identity_labels)

['woman', 'women', 'man', 'men', 'girl', 'girls', 'boy', 'boys', 'she', 'he', 'her', 'him', 'his', 'female', 'male', 'mother', 'father', 'sister', 'brother', 'daughter', 'son', 'feminine', 'masculine', 'androgynous', 'trans', 'transgender', 'transsexual', 'nonbinary', 'non-binary', 'two-spirit', 'hijra', 'genderqueer', 'black', 'asian', 'hispanic', 'white', 'african', 'american', 'latino', 'latina', 'caucasian', 'africans', 'middle-eastern', 'australian', 'australians', 'asians', 'european', 'europeans', 'chinese', 'indian', 'indonesian', 'brazilian', 'pakistani', 'bangladeshi', 'russian', 'nigerian', 'japanese', 'mexican', 'filipino ', 'vietnamese ', 'german', 'egyptian', 'ethiopian', 'turkish', 'iranian', 'thai', 'congolese', 'french', 'british ', 'italian', 'korean', 'burmese', 'canadian ', 'australian ', 'spanish', 'dutch', 'swiss', 'saudi', 'argentinian ', 'taiwanese ', 'swedish ', 'belgian', 'polish', 'israeli', 'irish', 'greek', 'ukrainian ', 'jamaican ', 'mongolian', 'armenian'

In [3]:
#Load SNLI dataset
import pandas as pd
snli_data = pd.read_json(path_or_buf="/content/drive/MyDrive/Colab Notebooks/ss-data/snli_1.0_train.jsonl", lines=True)
snli_data

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
1,[contradiction],3416050480.jpg#4,contradiction,3416050480.jpg#4r1c,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is at a diner, ordering an omelette.",( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
2,[entailment],3416050480.jpg#4,entailment,3416050480.jpg#4r1e,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is outdoors, on a horse.","( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
3,[neutral],2267923837.jpg#2,neutral,2267923837.jpg#2r1n,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,They are smiling at their parents,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...
4,[entailment],2267923837.jpg#2,entailment,2267923837.jpg#2r1e,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,There are children present,( There ( ( are children ) present ) ),(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...
...,...,...,...,...,...,...,...,...,...,...
550147,[contradiction],2267923837.jpg#3,contradiction,2267923837.jpg#3r1c,Four dirty and barefooted children.,( ( ( ( Four dirty ) and ) ( barefooted childr...,(ROOT (NP (NP (CD Four) (NNS dirty)) (CC and) ...,four kids won awards for 'cleanest feet',( ( four kids ) ( ( won awards ) ( ( ( for ` )...,(ROOT (S (NP (CD four) (NNS kids)) (VP (VBD wo...
550148,[neutral],2267923837.jpg#3,neutral,2267923837.jpg#3r1n,Four dirty and barefooted children.,( ( ( ( Four dirty ) and ) ( barefooted childr...,(ROOT (NP (NP (CD Four) (NNS dirty)) (CC and) ...,"four homeless children had their shoes stolen,...",( ( ( ( ( ( four ( homeless children ) ) ( had...,(ROOT (S (S (NP (CD four) (JJ homeless) (NNS c...
550149,[neutral],7979219683.jpg#2,neutral,7979219683.jpg#2r1n,A man is surfing in a bodysuit in beautiful bl...,( ( A man ) ( ( is ( surfing ( in ( ( a bodysu...,(ROOT (S (NP (DT A) (NN man)) (VP (VBZ is) (VP...,A man in a bodysuit is competing in a surfing ...,( ( ( A man ) ( in ( a bodysuit ) ) ) ( ( is (...,(ROOT (S (NP (NP (DT A) (NN man)) (PP (IN in) ...
550150,[contradiction],7979219683.jpg#2,contradiction,7979219683.jpg#2r1c,A man is surfing in a bodysuit in beautiful bl...,( ( A man ) ( ( is ( surfing ( in ( ( a bodysu...,(ROOT (S (NP (DT A) (NN man)) (VP (VBZ is) (VP...,A man in a business suit is heading to a board...,( ( ( A man ) ( in ( a ( business suit ) ) ) )...,(ROOT (S (NP (NP (DT A) (NN man)) (PP (IN in) ...


In [4]:
#De-duplicate
snli_data_sub = snli_data[['sentence1','sentence2']]
print(len(snli_data_sub))
snli_data_sub.drop_duplicates(subset=['sentence1', 'sentence2'], keep='last')
print(len(snli_data_sub))

550152
550152


**Data preparation**

In [15]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
nltk.download('punkt')

from nltk.corpus import stopwords
import string
nltk.download('stopwords')

from itertools import combinations
from tqdm import tqdm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
#util functions
stop_words = set(stopwords.words('english'))
string.punctuation = string.punctuation +'"'+'"'+'-'+'''+'''+'—'
string.punctuation
removal_list = list(stop_words) + list(string.punctuation)+ ['lt','rt']
removal_list

def remove_stopwords(sent_tokens):
    filtered_words = [word for word in sent_tokens if word not in removal_list]
    return filtered_words

In [7]:
#Lowercase
snli_data_sub['sentence1'] = snli_data_sub['sentence1'].map(str.lower)
snli_data_sub['sentence2'] = snli_data_sub['sentence2'].map(str.lower)
snli_data_sub

<ipython-input-7-e4d1754a863d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence1'] = snli_data_sub['sentence1'].map(str.lower)
<ipython-input-7-e4d1754a863d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence2'] = snli_data_sub['sentence2'].map(str.lower)


,sentence1,sentence2
0,a person on a horse jumps over a broken down a...,a person is training his horse for a competition.
1,a person on a horse jumps over a broken down a...,"a person is at a diner, ordering an omelette."
2,a person on a horse jumps over a broken down a...,"a person is outdoors, on a horse."
3,children smiling and waving at camera,they are smiling at their parents
4,children smiling and waving at camera,there are children present
...,...,...
550147,four dirty and barefooted children.,four kids won awards for 'cleanest feet'
550148,four dirty and barefooted children.,"four homeless children had their shoes stolen,..."
550149,a man is surfing in a bodysuit in beautiful bl...,a man in a bodysuit is competing in a surfing ...
550150,a man is surfing in a bodysuit in beautiful bl...,a man in a business suit is heading to a board...


In [8]:
#Tokenize
snli_data_sub['sentence1_token'] = snli_data_sub['sentence1'].apply(nltk.word_tokenize)
snli_data_sub['sentence2_token'] = snli_data_sub['sentence2'].apply(nltk.word_tokenize)

snli_data_sub

<ipython-input-8-d79b5798814b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence1_token'] = snli_data_sub['sentence1'].apply(nltk.word_tokenize)
<ipython-input-8-d79b5798814b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence2_token'] = snli_data_sub['sentence2'].apply(nltk.word_tokenize)


,sentence1,sentence2,sentence1_token,sentence2_token
0,a person on a horse jumps over a broken down a...,a person is training his horse for a competition.,"[a, person, on, a, horse, jumps, over, a, brok...","[a, person, is, training, his, horse, for, a, ..."
1,a person on a horse jumps over a broken down a...,"a person is at a diner, ordering an omelette.","[a, person, on, a, horse, jumps, over, a, brok...","[a, person, is, at, a, diner, ,, ordering, an,..."
2,a person on a horse jumps over a broken down a...,"a person is outdoors, on a horse.","[a, person, on, a, horse, jumps, over, a, brok...","[a, person, is, outdoors, ,, on, a, horse, .]"
3,children smiling and waving at camera,they are smiling at their parents,"[children, smiling, and, waving, at, camera]","[they, are, smiling, at, their, parents]"
4,children smiling and waving at camera,there are children present,"[children, smiling, and, waving, at, camera]","[there, are, children, present]"
...,...,...,...,...
550147,four dirty and barefooted children.,four kids won awards for 'cleanest feet',"[four, dirty, and, barefooted, children, .]","[four, kids, won, awards, for, 'cleanest, feet..."
550148,four dirty and barefooted children.,"four homeless children had their shoes stolen,...","[four, dirty, and, barefooted, children, .]","[four, homeless, children, had, their, shoes, ..."
550149,a man is surfing in a bodysuit in beautiful bl...,a man in a bodysuit is competing in a surfing ...,"[a, man, is, surfing, in, a, bodysuit, in, bea...","[a, man, in, a, bodysuit, is, competing, in, a..."
550150,a man is surfing in a bodysuit in beautiful bl...,a man in a business suit is heading to a board...,"[a, man, is, surfing, in, a, bodysuit, in, bea...","[a, man, in, a, business, suit, is, heading, t..."


In [9]:
#Remove stop-words
snli_data_sub['sentence1_token_nostopwords'] = snli_data_sub['sentence1_token'].apply(remove_stopwords)
snli_data_sub['sentence2_token_nostopwords'] = snli_data_sub['sentence2_token'].apply(remove_stopwords)

snli_data_sub

<ipython-input-9-eab9860fa455>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence1_token_nostopwords'] = snli_data_sub['sentence1_token'].apply(remove_stopwords)
<ipython-input-9-eab9860fa455>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence2_token_nostopwords'] = snli_data_sub['sentence2_token'].apply(remove_stopwords)


,sentence1,sentence2,sentence1_token,sentence2_token,sentence1_token_nostopwords,sentence2_token_nostopwords
0,a person on a horse jumps over a broken down a...,a person is training his horse for a competition.,"[a, person, on, a, horse, jumps, over, a, brok...","[a, person, is, training, his, horse, for, a, ...","[person, horse, jumps, broken, airplane]","[person, training, horse, competition]"
1,a person on a horse jumps over a broken down a...,"a person is at a diner, ordering an omelette.","[a, person, on, a, horse, jumps, over, a, brok...","[a, person, is, at, a, diner, ,, ordering, an,...","[person, horse, jumps, broken, airplane]","[person, diner, ordering, omelette]"
2,a person on a horse jumps over a broken down a...,"a person is outdoors, on a horse.","[a, person, on, a, horse, jumps, over, a, brok...","[a, person, is, outdoors, ,, on, a, horse, .]","[person, horse, jumps, broken, airplane]","[person, outdoors, horse]"
3,children smiling and waving at camera,they are smiling at their parents,"[children, smiling, and, waving, at, camera]","[they, are, smiling, at, their, parents]","[children, smiling, waving, camera]","[smiling, parents]"
4,children smiling and waving at camera,there are children present,"[children, smiling, and, waving, at, camera]","[there, are, children, present]","[children, smiling, waving, camera]","[children, present]"
...,...,...,...,...,...,...
550147,four dirty and barefooted children.,four kids won awards for 'cleanest feet',"[four, dirty, and, barefooted, children, .]","[four, kids, won, awards, for, 'cleanest, feet...","[four, dirty, barefooted, children]","[four, kids, awards, 'cleanest, feet]"
550148,four dirty and barefooted children.,"four homeless children had their shoes stolen,...","[four, dirty, and, barefooted, children, .]","[four, homeless, children, had, their, shoes, ...","[four, dirty, barefooted, children]","[four, homeless, children, shoes, stolen, feet..."
550149,a man is surfing in a bodysuit in beautiful bl...,a man in a bodysuit is competing in a surfing ...,"[a, man, is, surfing, in, a, bodysuit, in, bea...","[a, man, in, a, bodysuit, is, competing, in, a...","[man, surfing, bodysuit, beautiful, blue, water]","[man, bodysuit, competing, surfing, competition]"
550150,a man is surfing in a bodysuit in beautiful bl...,a man in a business suit is heading to a board...,"[a, man, is, surfing, in, a, bodysuit, in, bea...","[a, man, in, a, business, suit, is, heading, t...","[man, surfing, bodysuit, beautiful, blue, water]","[man, business, suit, heading, board, meeting]"


In [10]:
print(len(snli_data_sub))
snli_data_sub['sentence1_token_nostopwords_str'] = snli_data_sub['sentence1_token_nostopwords'].apply(lambda x: ' '.join(x))
snli_data_sub['sentence2_token_nostopwords_str'] = snli_data_sub['sentence2_token_nostopwords'].apply(lambda x: ' '.join(x))
snli_data_sub.drop_duplicates(subset=['sentence1_token_nostopwords_str', 'sentence2_token_nostopwords_str'], keep='last', inplace=True)
print(len(snli_data_sub))

550152


<ipython-input-10-f92728b880f2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence1_token_nostopwords_str'] = snli_data_sub['sentence1_token_nostopwords'].apply(lambda x: ' '.join(x))
<ipython-input-10-f92728b880f2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence2_token_nostopwords_str'] = snli_data_sub['sentence2_token_nostopwords'].apply(lambda x: ' '.join(x))


547562


<ipython-input-10-f92728b880f2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub.drop_duplicates(subset=['sentence1_token_nostopwords_str', 'sentence2_token_nostopwords_str'], keep='last', inplace=True)


**Word association analysis**
```latex
PMI(w_i, w_j) = log_2 \frac{p(w_i, w_j)}{P(w_i)P(w_j)} = log_2\frac{N\cdot c(w_i, w_j)}{c(w_i)c(w_j)}
```


In [11]:
#Computing Unigram frequency
corpus = []
for sent in (snli_data_sub['sentence1_token_nostopwords'].tolist() + snli_data_sub['sentence2_token_nostopwords'].tolist()):
  corpus+=(sent)
unigram_frequency = Counter(corpus)
unigram_frequency.most_common(20)

[('man', 265138),
 ('woman', 137105),
 ('two', 121714),
 ('people', 120802),
 ('wearing', 80592),
 ('young', 61353),
 ('men', 60820),
 ('playing', 59220),
 ('girl', 59094),
 ('boy', 58041),
 ('white', 56785),
 ('shirt', 56204),
 ('black', 54824),
 ('dog', 53690),
 ('sitting', 53544),
 ('blue', 49040),
 ('standing', 46189),
 ('red', 43129),
 ('group', 43057),
 ('walking', 38678)]

In [12]:
#Remove words with less than 15 freq
from itertools import dropwhile
print(len(unigram_frequency))
for key, count in dropwhile(lambda key_count: key_count[1] >= 15, unigram_frequency.most_common()):
    del unigram_frequency[key]
print(len(unigram_frequency))

36420
10662


In [13]:
snli_data_sub['sentence1_sentence2_token_nostopwords'] = snli_data_sub['sentence1_token_nostopwords'] + snli_data_sub['sentence2_token_nostopwords']

def f7(seq):#deduplicate-without changing order in a list
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

snli_data_sub['sentence1_sentence2_token_nostopwords_dedup'] = snli_data_sub['sentence1_sentence2_token_nostopwords'].apply(f7)


<ipython-input-13-8eb377f044f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence1_sentence2_token_nostopwords'] = snli_data_sub['sentence1_token_nostopwords'] + snli_data_sub['sentence2_token_nostopwords']
<ipython-input-13-8eb377f044f6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence1_sentence2_token_nostopwords_dedup'] = snli_data_sub['sentence1_sentence2_token_nostopwords'].apply(f7)


In [16]:
bigram_doc=[]
for ele in tqdm(snli_data_sub['sentence1_sentence2_token_nostopwords_dedup']):
  bigram_doc.append(list(set(list(combinations(ele, 2)))))

bigram=[]
for doc in bigram_doc:
  bigram += doc
bigram_frequency = Counter(bigram)
bigram_frequency

100%|██████████| 547562/547562 [00:10<00:00, 52451.30it/s]


Counter({('broken', 'training'): 1,
         ('person', 'jumps'): 263,
         ('training', 'competition'): 36,
         ('horse', 'training'): 7,
         ('person', 'competition'): 106,
         ('horse', 'broken'): 5,
         ('jumps', 'competition'): 65,
         ('person', 'training'): 18,
         ('person', 'horse'): 216,
         ('airplane', 'competition'): 2,
         ('person', 'broken'): 29,
         ('jumps', 'training'): 10,
         ('broken', 'airplane'): 4,
         ('jumps', 'broken'): 7,
         ('horse', 'airplane'): 23,
         ('airplane', 'training'): 1,
         ('horse', 'jumps'): 91,
         ('broken', 'competition'): 1,
         ('horse', 'competition'): 126,
         ('person', 'airplane'): 37,
         ('jumps', 'airplane'): 6,
         ('broken', 'ordering'): 1,
         ('horse', 'ordering'): 1,
         ('person', 'diner'): 5,
         ('ordering', 'omelette'): 1,
         ('person', 'omelette'): 1,
         ('jumps', 'diner'): 1,
         ('diner',

In [17]:
#Remove words-pair with less than 10 freq
from itertools import dropwhile
print(len(bigram_frequency))
for key, count in dropwhile(lambda key_count: key_count[1] >= 10, bigram_frequency.most_common()):
    del bigram_frequency[key]
print(len(bigram_frequency))

3670850
385251


In [18]:
#Calculate PMI
import math

def pmi(word1, word2, unigram_freq, bigram_freq):
  if word1 in unigram_freq.keys() and word2 in unigram_freq.keys():
    if (word1, word2) in bigram_freq.keys() or (word2, word1) in bigram_freq.keys():
      prob_word1 = unigram_freq[word1] / float(sum(unigram_freq.values()))
      prob_word2 = unigram_freq[word2] / float(sum(unigram_freq.values()))
      prob_word1_word2 = (bigram_freq[(word1, word2)]+bigram_freq[(word2, word1)]) / float(sum(bigram_freq.values()))
      if prob_word1_word2 >0:
        return math.log(prob_word1_word2/float(prob_word1*prob_word2),2)


In [ ]:
identity_labels

['woman',
 'women',
 'man',
 'men',
 'girl',
 'girls',
 'boy',
 'boys',
 'she',
 'he',
 'her',
 'him',
 'his',
 'female',
 'male',
 'mother',
 'father',
 'sister',
 'brother',
 'daughter',
 'son',
 'feminine',
 'masculine',
 'androgynous',
 'trans',
 'transgender',
 'transsexual',
 'nonbinary',
 'non-binary',
 'two-spirit',
 'hijra',
 'genderqueer',
 'black',
 'asian',
 'hispanic',
 'white',
 'african',
 'american',
 'latino',
 'latina',
 'caucasian',
 'africans',
 'middle-eastern',
 'australian',
 'australians',
 'asians',
 'european',
 'europeans',
 'chinese',
 'indian',
 'indonesian',
 'brazilian',
 'pakistani',
 'bangladeshi',
 'russian',
 'nigerian',
 'japanese',
 'mexican',
 'filipino ',
 'vietnamese ',
 'german',
 'egyptian',
 'ethiopian',
 'turkish',
 'iranian',
 'thai',
 'congolese',
 'french',
 'british ',
 'italian',
 'korean',
 'burmese',
 'canadian ',
 'australian ',
 'spanish',
 'dutch',
 'swiss',
 'saudi',
 'argentinian ',
 'taiwanese ',
 'swedish ',
 'belgian',
 'polish

In [19]:
def Sort(tuple):
    # reverse = None (Sorts in Ascending order)
    return(sorted(tuple, key = lambda a: a[1], reverse = True))

def get_pmi(identity_label,unigram_frequency,bigram_frequency):
  pmi_identity_label=[]
  for word in tqdm(unigram_frequency.keys()):
    pmi_score = pmi(word1=identity_label.lower(), word2=word,unigram_freq=unigram_frequency,bigram_freq=bigram_frequency)
    if pmi_score:
      pmi_identity_label.append((word,pmi_score))

  return Sort(pmi_identity_label)


In [20]:
male_pmi_data = get_pmi('man',unigram_frequency,bigram_frequency)
female_pmi_data = get_pmi('woman',unigram_frequency,bigram_frequency)
gay_pmi_data = get_pmi('gay',unigram_frequency,bigram_frequency)

100%|██████████| 10662/10662 [00:00<00:00, 19313.65it/s]


In [ ]:
search_term = "marble-looking"
mask = snli_data_sub["sentence1"].str.contains(search_term, case=False, na=False)
pd.set_option('display.max_colwidth', 0)
snli_data_sub[mask][["sentence1","sentence2"]]

In [22]:
from collections import defaultdict as dd
pmi_identity_label = dd(list)
for identity_label in (identity_labels):
  print(identity_label)
  for word in tqdm(unigram_frequency.keys()):
    pmi_score = pmi(word1=identity_label, word2=word,unigram_freq=unigram_frequency,bigram_freq=bigram_frequency)
    #print("PMI:",pmi_score)
    if pmi_score:
      pmi_identity_label[identity_label].append((word,pmi_score))

woman


100%|██████████| 10662/10662 [01:28<00:00, 120.09it/s]


women


100%|██████████| 10662/10662 [00:37<00:00, 287.54it/s] 


man


...

teenager


100%|██████████| 10662/10662 [00:02<00:00, 4935.85it/s]


teenagers


100%|██████████| 10662/10662 [00:02<00:00, 4508.48it/s]


100%|██████████| 10662/10662 [00:00<00:00, 701705.15it/s]


In [23]:
for identity in pmi_identity_label.keys():
    pmi_identity_label[identity] = Sort(pmi_identity_label[identity])


In [25]:
for key in pmi_identity_label.keys():
    print(key, pmi_identity_label[key][:20])

woman [('marble-looking', 3.9246229431677464), ('edwards', 3.9246229431677464), ('bandidos', 3.9246229431677464), ('leaf-strewn', 3.9246229431677464), ('leaf-lined', 3.9246229431677464), ('leave-like', 3.9246229431677464), ('mirthlessly', 3.9246229431677464), ('coke-a-cola', 3.9246229431677464), ('ruts', 3.9246229431677464), ('wearubg', 3.9246229431677464), ('tableau', 3.9246229431677464), ('radishes', 3.9246229431677464), ('pull-overs', 3.9246229431677464), ('denomination', 3.9246229431677464), ('t-short', 3.9246229431677464), ('lavendar', 3.9246229431677464), ('buzzes', 3.9246229431677464), ('homemade-looking', 3.9246229431677464), ('event-', 3.9246229431677464), ('spheres', 3.9246229431677464)]
women [('disbelieving', 5.925517637795422), ('marley', 5.925517637795422), ('partially-drunk', 5.925517637795422), ('futon-style', 5.925517637795422), ('footrest', 5.925517637795422), ('hand-crafting', 5.925517637795422), ('headwraps', 5.861387300375705), ('blanked', 5.832408233403941), ('wel

In [27]:
search_term = "extravagant"#wolf-like"#slightly-untidy"
mask = snli_data_sub["sentence1"].str.contains(search_term, case=False, na=False)
pd.set_option('display.max_colwidth', 0)
snli_data_sub[mask][["sentence1","sentence2"]]

,sentence1,sentence2
20205,a indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,the woman is preparing for her wedding.
20206,a indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,"a woman dresses in western fashion, with unadorned fingers and head."
20207,a indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,an indian lady is wearing a beautiful outfit.
20208,a indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,an indian woman is wearing traditional styles.
20209,a indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,a woman shows off to a crowd of people.
20210,a indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,a woman is ready for a national cultural celebration.
20211,a indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,a japanese woman is wearing a kimono.
20212,a indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,"a very colorfully robed female with fingers painted red, who's culture appears to be indian, has on an elaborate head piece."
20213,a indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,"a woman is dressed in traditional, ethnic clothing."
20214,a indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,a woman wearing an old-fashioned apron holds an apple pie in her hands.
